# VisualCLA Inference & Deployment

该notebook展示了从环境准备、模型合并到推理以及Gradio部署的操作流程。

⚠️以下代码需消耗 20G+的内存，请确保刷出来的机器RAM满足要求。

## 一、环境与代码准备工作

### 1.1 安装相关依赖

In [ ]:
!pip install transformers==4.30.2
!pip install peft==0.3.0
!pip install sentencepiece
!pip install bitsandbytes
!pip install gradio
!pip install mdtex2html

### 1.2 克隆目录和代码

In [ ]:
!git clone https://github.com/ymcui/Chinese-LLaMA-Alpaca

In [ ]:
!git clone https://github.com/airaria/Visual-Chinese-LLaMA-Alpaca

### 1.3 安装VisualCLA代码

In [ ]:
!pip install -e Visual-Chinese-LLaMA-Alpaca/

## 二、合并模型

合并模型过程分为两步：
* 从原版LLaMA合并出Chinese-LLaMA-Alpaca-Plus-7B模型
* 从Chinese-LLaMA-Alpaca-Plus-7B合并出VisualCLA模型

### 2.1 合并Chinese-Alpaca-Plus-7B模型

此处使用的是🤗模型库中提供的基模型（HF格式）
- 基模型：`elinas/llama-7b-hf-transformers-4.29` *（use at your own risk，我们比对过SHA256和正版一致，但你应确保自己有权使用该模型）*
- LoRA模型：'ziqingyang/chinese-llama-plus-lora-7b,ziqingyang/chinese-alpaca-plus-lora-7b'

转换好的模型存放在`chinese-alpaca-plus-7b`目录。


In [ ]:
!python ./Chinese-LLaMA-Alpaca/scripts/merge_llama_with_chinese_lora_low_mem.py \
    --base_model 'elinas/llama-7b-hf-transformers-4.29' \
    --lora_model 'ziqingyang/chinese-llama-plus-lora-7b,ziqingyang/chinese-alpaca-plus-lora-7b' \
    --output_type huggingface \
    --output_dir chinese-alpaca-plus-7b

### 2.2 合并VisualCLA模型

基于Chinese-Alpaca-Plus-7B和CLIP-ViT，合并得到最终的VisualCLA模型。

转换好的模型存放在`visualcla`目录

In [ ]:
! python Visual-Chinese-LLaMA-Alpaca/scripts/merge_llama_with_visualcla_lora.py \
    --text_model chinese-alpaca-plus-7b \
    --vision_model openai/clip-vit-large-patch14 \
    --lora_model ziqingyang/visualcla-7b-v0.1 \
    --output_dir visualcla

## 三、 模型推理与部署

合并完成后，用户可以选择是进行命令行的推理或启动Gradio demo进行可视化体验

### 3.1 命令行推理

运行下面的命令将启动交互式命令行程序进行推理。

默认读入image_file文件，用户可输入向模型提问题或输入指令。

也可通过change image命令更换图片。详细使用说明可参见启动后的提示信息。

In [ ]:
!python Visual-Chinese-LLaMA-Alpaca/scripts/inference/inference.py \
    --visualcla_model visualcla \
    --image_file Visual-Chinese-LLaMA-Alpaca/pics/examples/food.jpg \
    --load_in_8bit

### 3.2 启动Gradio

运行下面的命令启动Gradio demo。

成功启动后单元格中将显示url地址。打开url地址即可访问demo。

In [ ]:
!python Visual-Chinese-LLaMA-Alpaca/scripts/inference/gradio_demo.py --visualcla_model visualcla --load_in_8bit --share